In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('FluPrediction').getOrCreate()

In [2]:
Flu = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("Data/FluView.csv")

Area = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("Data/Area.csv")
Income = spark.read.format("csv").option("header", "true").load("Data/Income.csv")
Health = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("Data/HealthCare.csv")
Virus = spark.read.format("csv").option("header", "true").load("Data/typeVirus.csv")
Population = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load("Data/population.csv")

In [3]:
Flu.show(n=5) 
Flu.columns

+---------+----------------+-----------+----+-------+
|STATENAME|ACTIVITYESTIMATE|    WEEKEND|WEEK| SEASON|
+---------+----------------+-----------+----+-------+
|  Alabama|     No Activity|Oct-04-2003|  40|2003-04|
|  Alabama|     No Activity|Oct-11-2003|  41|2003-04|
|  Alabama|     No Activity|Oct-18-2003|  42|2003-04|
|  Alabama|  Local Activity|Oct-25-2003|  43|2003-04|
|  Alabama|        Sporadic|Nov-01-2003|  44|2003-04|
+---------+----------------+-----------+----+-------+
only showing top 5 rows



['STATENAME', 'ACTIVITYESTIMATE', 'WEEKEND', 'WEEK', 'SEASON']

In [4]:
Area.show(n=5) 
Area.columns

+----------+-------+
| STATENAME|   AREA|
+----------+-------+
|   Alabama| 135767|
|    Alaska|1723337|
|   Arizona| 295234|
|  Arkansas| 137732|
|California| 423967|
+----------+-------+
only showing top 5 rows



['STATENAME', 'AREA']

In [5]:
Income.show(n=5) 
Income.columns

+---+----------+------+-------+
|_c0| STATENAME|INCOME| SEASON|
+---+----------+------+-------+
|  2|   Alabama| 51113|2017-18|
|  3|    Alaska| 72231|2017-18|
|  4|   Arizona| 61125|2017-18|
|  5|  Arkansas| 48829|2017-18|
|  6|California| 69759|2017-18|
+---+----------+------+-------+
only showing top 5 rows



['_c0', 'STATENAME', 'INCOME', 'SEASON']

In [6]:
Health.show(n=5) 
Health.columns

+----------+-----------+--------------+--------------+-----------------+
| STATENAME|All persons|Under 65 years|Under 18 years|65 years and over|
+----------+-----------+--------------+--------------+-----------------+
|   Alabama|       84,6|          82,3|          91,1|             98,5|
|    Alaska|       85,2|          82,9|          91,8|             99,8|
|   Arizona|         81|          79,3|          83,5|             96,5|
|  Arkansas|         82|          79,3|          86,3|             97,3|
|California|       81,6|          78,6|          91,7|               98|
+----------+-----------+--------------+--------------+-----------------+
only showing top 5 rows



['STATENAME',
 'All persons',
 'Under 65 years',
 'Under 18 years',
 '65 years and over']

In [7]:
Virus.show(n=5) 
Virus.columns

+---+----+-------+---+-----+---+---+---+---+----+---+---+
|_c0|WEEK| SEASON|AH3|AH1N1|  A|BVL|BYL|  B|H3N2| AU|AH1|
+---+----+-------+---+-----+---+---+---+---+----+---+---+
|  1|  40|2018-19| 11|   29|  8|  7| 11|  0|   0|  0|  0|
|  2|  41|2018-19| 11|   49|  8|  3|  5|  5|   0|  0|  0|
|  3|  42|2018-19| 18|   48| 13|  1| 12|  1|   0|  0|  0|
|  4|  43|2018-19| 33|   73|  2|  0| 12|  2|   0|  0|  0|
|  5|  44|2018-19| 20|   77|  3|  1|  8|  1|   0|  0|  0|
+---+----+-------+---+-----+---+---+---+---+----+---+---+
only showing top 5 rows



['_c0',
 'WEEK',
 'SEASON',
 'AH3',
 'AH1N1',
 'A',
 'BVL',
 'BYL',
 'B',
 'H3N2',
 'AU',
 'AH1']

In [8]:
Population.show(n=5) 
Population.columns

+----------+--------+-------+
| STATENAME|     POP| SEASON|
+----------+--------+-------+
|   Alabama| 4503491|2003-04|
|    Alaska|  648414|2003-04|
|   Arizona| 5510364|2003-04|
|  Arkansas| 2724816|2003-04|
|California|35253159|2003-04|
+----------+--------+-------+
only showing top 5 rows



['STATENAME', 'POP', 'SEASON']

In [9]:
Flu.describe().show()
Area.describe().show()
Income.describe().show()
Health.describe().show()
Virus.describe().show()
Population.describe().show()

+-------+---------+----------------+-----------+------------------+-------+
|summary|STATENAME|ACTIVITYESTIMATE|    WEEKEND|              WEEK| SEASON|
+-------+---------+----------------+-----------+------------------+-------+
|  count|    29104|           29104|      29104|             29104|  29104|
|   mean|     null|            null|       null|24.787073941726224|   null|
| stddev|     null|            null|       null| 17.88996084805653|   null|
|    min|  Alabama|  Local Activity|Apr-01-2006|                 1|2003-04|
|    max|  Wyoming|      Widespread|Sep-26-2009|                 9|2018-19|
+-------+---------+----------------+-----------+------------------+-------+

+-------+---------+------------------+
|summary|STATENAME|              AREA|
+-------+---------+------------------+
|  count|       57|                57|
|   mean|     null|154560.36842105264|
| stddev|     null|248208.14633716136|
|    min|  Alabama|            104656|
|    max|  Wyoming|              9998|
+--

In [10]:
Flu.printSchema()
Area.printSchema()
Income.printSchema()
Health.printSchema()
Virus.printSchema()
Population.printSchema()

root
 |-- STATENAME: string (nullable = true)
 |-- ACTIVITYESTIMATE: string (nullable = true)
 |-- WEEKEND: string (nullable = true)
 |-- WEEK: string (nullable = true)
 |-- SEASON: string (nullable = true)

root
 |-- STATENAME: string (nullable = true)
 |-- AREA: string (nullable = true)

root
 |-- _c0: string (nullable = true)
 |-- STATENAME: string (nullable = true)
 |-- INCOME: string (nullable = true)
 |-- SEASON: string (nullable = true)

root
 |-- STATENAME: string (nullable = true)
 |-- All persons: string (nullable = true)
 |-- Under 65 years: string (nullable = true)
 |-- Under 18 years: string (nullable = true)
 |-- 65 years and over: string (nullable = true)

root
 |-- _c0: string (nullable = true)
 |-- WEEK: string (nullable = true)
 |-- SEASON: string (nullable = true)
 |-- AH3: string (nullable = true)
 |-- AH1N1: string (nullable = true)
 |-- A: string (nullable = true)
 |-- BVL: string (nullable = true)
 |-- BYL: string (nullable = true)
 |-- B: string (nullable = true

In [11]:
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType,FloatType)

In [12]:
data_schema = [StructField('STATENAME',StringType(),True),
              StructField('ACTIVITYESTIMATE',StringType(),True),
              StructField('WEEKEND',StringType(),True),
              StructField('WEEK',IntegerType(),True),
              StructField('SEASON',StringType(),True)]

final_struct = StructType(fields=data_schema)

Flu = spark.read.format("csv").option("header", "true").option("delimiter", ";").schema(final_struct).load("Data/FluView.csv") 
Flu = Flu.drop('WEEKEND')
Flu.printSchema()

root
 |-- STATENAME: string (nullable = true)
 |-- ACTIVITYESTIMATE: string (nullable = true)
 |-- WEEK: integer (nullable = true)
 |-- SEASON: string (nullable = true)



In [13]:
data_schema_Area = [StructField('STATENAME',StringType(),True),
              StructField('AREA',IntegerType(),True)]

final_struct_Area = StructType(fields=data_schema_Area)

Area = spark.read.format("csv").option("header", "true").option("delimiter", "\t").schema(final_struct_Area).load("Data/Area.csv") 

Area.printSchema()
Area.show(n=5) 


root
 |-- STATENAME: string (nullable = true)
 |-- AREA: integer (nullable = true)

+----------+-------+
| STATENAME|   AREA|
+----------+-------+
|   Alabama| 135767|
|    Alaska|1723337|
|   Arizona| 295234|
|  Arkansas| 137732|
|California| 423967|
+----------+-------+
only showing top 5 rows



In [14]:
data_schema_income = [StructField('_c0',StringType(),True),
              StructField('STATENAME',StringType(),True),
              StructField('INCOME',IntegerType(),True),
              StructField('SEASON',StringType(),True)]

final_struct_income = StructType(fields=data_schema_income)

Income = spark.read.format("csv").option("header", "true").schema(final_struct_income).load("Data/Income.csv") 
Income = Income.drop('_c0')
Income.printSchema()
Income.show(n=5) 

root
 |-- STATENAME: string (nullable = true)
 |-- INCOME: integer (nullable = true)
 |-- SEASON: string (nullable = true)

+----------+------+-------+
| STATENAME|INCOME| SEASON|
+----------+------+-------+
|   Alabama| 51113|2017-18|
|    Alaska| 72231|2017-18|
|   Arizona| 61125|2017-18|
|  Arkansas| 48829|2017-18|
|California| 69759|2017-18|
+----------+------+-------+
only showing top 5 rows



In [15]:
Health = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("Data/HealthCare.csv")

from pyspark.sql.functions import *
Health = Health.withColumn('65 years and over',regexp_replace('65 years and over', ',', '.').cast('float'))
Health = Health.withColumn('Under 65 years',regexp_replace('Under 65 years', ',', '.').cast('float'))
Health = Health.withColumn('Under 18 years',regexp_replace('Under 18 years', ',', '.').cast('float'))
Health = Health.withColumn('All persons',regexp_replace('All persons', ',', '.').cast('float'))

Health.printSchema()

Health = Health.withColumnRenamed('All persons', 'HealthCare_All')
Health = Health.withColumnRenamed('Under 65 years', 'HealthCare_Under65')
Health = Health.withColumnRenamed('Under 18 years', 'HealthCare_Under18')
Health = Health.withColumnRenamed('65 years and over', 'HealthCare_Over65')

Health.show(n=5)

root
 |-- STATENAME: string (nullable = true)
 |-- All persons: float (nullable = true)
 |-- Under 65 years: float (nullable = true)
 |-- Under 18 years: float (nullable = true)
 |-- 65 years and over: float (nullable = true)

+----------+--------------+------------------+------------------+-----------------+
| STATENAME|HealthCare_All|HealthCare_Under65|HealthCare_Under18|HealthCare_Over65|
+----------+--------------+------------------+------------------+-----------------+
|   Alabama|          84.6|              82.3|              91.1|             98.5|
|    Alaska|          85.2|              82.9|              91.8|             99.8|
|   Arizona|          81.0|              79.3|              83.5|             96.5|
|  Arkansas|          82.0|              79.3|              86.3|             97.3|
|California|          81.6|              78.6|              91.7|             98.0|
+----------+--------------+------------------+------------------+-----------------+
only showing top 

In [16]:
#data_schema_virus = [StructField('_c0',StringType(),True),
#              StructField('WEEK',IntegerType(),True),
#              StructField('SEASON',StringType(),True),
#              StructField('AH3',IntegerType(),True),
#              StructField('AH1N1',IntegerType(),True),
#              StructField('A',IntegerType(),True),
#              StructField('BVL',IntegerType(),True),
#              StructField('BYL',IntegerType(),True),
#              StructField('B',IntegerType(),True),
#              StructField('H3N2',IntegerType(),True),
#              StructField('AU',IntegerType(),True),
#              StructField('AH1',IntegerType(),True)]

#final_struct_virus = StructType(fields=data_schema_virus)
#Virus = spark.read.format("csv").option("header", "true").schema(final_struct_virus).load("Data/typeVirus.csv")
#Virus.describe().show()
#Virus.printSchema()

Virus = spark.read.csv("Data/typeVirus.csv", header=True, inferSchema=True)

Virus = Virus.withColumn('AH1N1',Virus['AH1N1'].cast('integer'))
Virus = Virus.withColumn('BVL',Virus['BVL'].cast('integer'))
Virus = Virus.withColumn('BYL',Virus['BYL'].cast('integer'))
Virus.printSchema()
Virus.describe().show()



Virus = Virus.withColumnRenamed('AH3', 'TypeA_subH3')
Virus = Virus.withColumnRenamed('AH1N1', 'TypeA_subH1N1')
Virus = Virus.withColumnRenamed('A', 'TypeA')
Virus = Virus.withColumnRenamed('AU', 'TypeA_subU')
Virus = Virus.withColumnRenamed('AH1', 'TypeA_subH1')
Virus = Virus.withColumnRenamed('H3N2', 'TypeA_subH3N2')
Virus = Virus.withColumnRenamed('B', 'TypeB')
Virus = Virus.withColumnRenamed('BVL', 'TypeB_subVL')
Virus = Virus.withColumnRenamed('BYL', 'TypeB_subYL')

#Virus.show(n=5)


root
 |-- _c0: integer (nullable = true)
 |-- WEEK: integer (nullable = true)
 |-- SEASON: string (nullable = true)
 |-- AH3: integer (nullable = true)
 |-- AH1N1: integer (nullable = true)
 |-- A: integer (nullable = true)
 |-- BVL: integer (nullable = true)
 |-- BYL: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- H3N2: integer (nullable = true)
 |-- AU: integer (nullable = true)
 |-- AH1: integer (nullable = true)

+-------+-----------------+------------------+-------+------------------+------------------+------------------+-----------------+------------------+-----------------+-------------------+------------------+------------------+
|summary|              _c0|              WEEK| SEASON|               AH3|             AH1N1|                 A|              BVL|               BYL|                B|               H3N2|                AU|               AH1|
+-------+-----------------+------------------+-------+------------------+------------------+------------------+

In [17]:
data_schema_pop = [StructField('STATENAME',StringType(),True),
              StructField('POP',IntegerType(),True),
              StructField('SEASON',StringType(),True)]

final_struct_pop = StructType(fields=data_schema_pop)

Population = spark.read.format("csv").option("header", "true").option("delimiter", "\t").schema(final_struct_pop).load("Data/population.csv")

Population.printSchema()
Population.show(n=5)

root
 |-- STATENAME: string (nullable = true)
 |-- POP: integer (nullable = true)
 |-- SEASON: string (nullable = true)

+----------+--------+-------+
| STATENAME|     POP| SEASON|
+----------+--------+-------+
|   Alabama| 4503491|2003-04|
|    Alaska|  648414|2003-04|
|   Arizona| 5510364|2003-04|
|  Arkansas| 2724816|2003-04|
|California|35253159|2003-04|
+----------+--------+-------+
only showing top 5 rows



In [18]:
Virus = Virus.withColumn('B',Virus['TypeB']+Virus['TypeB_subYL']+Virus['TypeB_subVL'])
#Virus.show(5)


In [19]:
Virus = Virus.drop('TypeB_subVL')
Virus = Virus.drop('TypeB_subYL')
Virus = Virus.drop('_c0')
Virus = Virus.drop('TypeB')
Virus = Virus.withColumnRenamed('B', 'TypeB')
Virus = Virus.na.fill(0, subset=['TypeB'])
Virus.show(5)
Virus.printSchema()
Virus.describe().show()

+----+-------+-----------+-------------+-----+-------------+----------+-----------+-----+
|WEEK| SEASON|TypeA_subH3|TypeA_subH1N1|TypeA|TypeA_subH3N2|TypeA_subU|TypeA_subH1|TypeB|
+----+-------+-----------+-------------+-----+-------------+----------+-----------+-----+
|  40|2018-19|         11|           29|    8|            0|         0|          0|   18|
|  41|2018-19|         11|           49|    8|            0|         0|          0|   13|
|  42|2018-19|         18|           48|   13|            0|         0|          0|   14|
|  43|2018-19|         33|           73|    2|            0|         0|          0|   14|
|  44|2018-19|         20|           77|    3|            0|         0|          0|   10|
+----+-------+-----------+-------------+-----+-------------+----------+-----------+-----+
only showing top 5 rows

root
 |-- WEEK: integer (nullable = true)
 |-- SEASON: string (nullable = true)
 |-- TypeA_subH3: integer (nullable = true)
 |-- TypeA_subH1N1: integer (nullable = t

### Merging

In [20]:
traintest = Flu.join(Population, (Flu.SEASON == Population.SEASON) & (Flu.STATENAME == Population.STATENAME) ).drop(Population.STATENAME).drop(Population.SEASON)
traintest = traintest.join(Area,(traintest.STATENAME == Area.STATENAME) ).drop(Area.STATENAME)
traintest = traintest.join(Health, (traintest.STATENAME == Health.STATENAME) ).drop(Health.STATENAME)
traintest = traintest.join(Income, (traintest.SEASON == Income.SEASON) & (traintest.STATENAME == Income.STATENAME) ).drop(Income.STATENAME).drop(Income.SEASON)
traintest = traintest.join(Virus, (traintest.SEASON == Virus.SEASON) & (traintest.WEEK == Virus.WEEK)).drop(Virus.WEEK).drop(Virus.SEASON)

traintest.show(5)
#traintest.describe().show()

+---------+----------------+----+-------+-------+------+--------------+------------------+------------------+-----------------+------+-----------+-------------+-----+-------------+----------+-----------+-----+
|STATENAME|ACTIVITYESTIMATE|WEEK| SEASON|    POP|  AREA|HealthCare_All|HealthCare_Under65|HealthCare_Under18|HealthCare_Over65|INCOME|TypeA_subH3|TypeA_subH1N1|TypeA|TypeA_subH3N2|TypeA_subU|TypeA_subH1|TypeB|
+---------+----------------+----+-------+-------+------+--------------+------------------+------------------+-----------------+------+-----------+-------------+-----+-------------+----------+-----------+-----+
|  Alabama|        Regional|  35|2009-10|4757938|135767|          84.6|              82.3|              91.1|             98.5| 39980|         12|            6|    0|            0|         0|          0|    0|
|  Alabama|      Widespread|  36|2009-10|4757938|135767|          84.6|              82.3|              91.1|             98.5| 39980|         14|            8|

In [21]:
traintest.printSchema()

root
 |-- STATENAME: string (nullable = true)
 |-- ACTIVITYESTIMATE: string (nullable = true)
 |-- WEEK: integer (nullable = true)
 |-- SEASON: string (nullable = true)
 |-- POP: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- HealthCare_All: float (nullable = true)
 |-- HealthCare_Under65: float (nullable = true)
 |-- HealthCare_Under18: float (nullable = true)
 |-- HealthCare_Over65: float (nullable = true)
 |-- INCOME: integer (nullable = true)
 |-- TypeA_subH3: integer (nullable = true)
 |-- TypeA_subH1N1: integer (nullable = true)
 |-- TypeA: integer (nullable = true)
 |-- TypeA_subH3N2: integer (nullable = true)
 |-- TypeA_subU: integer (nullable = true)
 |-- TypeA_subH1: integer (nullable = true)
 |-- TypeB: integer (nullable = true)



In [22]:
traintest.toPandas().to_csv('traintest.csv')

In [2]:
traintest = spark.read.csv("traintest.csv", header=True, inferSchema=True)

In [3]:
traintest.count()

17124

### Features creation

In [3]:
#prevweek
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window

traintest = spark.read.csv("traintest.csv", header=True, inferSchema=True)

w = Window().partitionBy().orderBy(col("_c0"))
traintest = traintest.select("*", lag("ACTIVITYESTIMATE").over(w).alias("PrevWeek"))#.na.drop()
# supprimer week 40
traintest = traintest.filter("WEEK != 40").select('*')

#traintest.show(2)
traintest.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- STATENAME: string (nullable = true)
 |-- ACTIVITYESTIMATE: string (nullable = true)
 |-- WEEK: integer (nullable = true)
 |-- SEASON: string (nullable = true)
 |-- POP: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- HealthCare_All: double (nullable = true)
 |-- HealthCare_Under65: double (nullable = true)
 |-- HealthCare_Under18: double (nullable = true)
 |-- HealthCare_Over65: double (nullable = true)
 |-- INCOME: integer (nullable = true)
 |-- TypeA_subH3: integer (nullable = true)
 |-- TypeA_subH1N1: integer (nullable = true)
 |-- TypeA: integer (nullable = true)
 |-- TypeA_subH3N2: integer (nullable = true)
 |-- TypeA_subU: integer (nullable = true)
 |-- TypeA_subH1: integer (nullable = true)
 |-- TypeB: integer (nullable = true)
 |-- PrevWeek: string (nullable = true)



In [4]:
#the number of states that was in each activity class during the previous week. This new feature are named NRegional, NLocalAct, NSporadic, NWidespread.

from pyspark.sql.functions import count

CSpo = traintest.where(traintest.PrevWeek=="Sporadic").groupBy("SEASON","WEEK").count().withColumnRenamed('count', 'NSporadic').withColumnRenamed('SEASON', 'SEASON2').withColumnRenamed('WEEK', 'WEEK2')
CLoc = traintest.where(traintest.PrevWeek=="Local Activity").groupBy("SEASON","WEEK").count().withColumnRenamed('count', 'NLocalAct').withColumnRenamed('SEASON', 'SEASON2').withColumnRenamed('WEEK', 'WEEK2')
CNoAct = traintest.where(traintest.PrevWeek=="No Activity").groupBy("SEASON","WEEK").count().withColumnRenamed('count', 'NNoAct').withColumnRenamed('SEASON', 'SEASON2').withColumnRenamed('WEEK', 'WEEK2')
CReg = traintest.where(traintest.PrevWeek=="Regional").groupBy("SEASON","WEEK").count().withColumnRenamed('count', 'NRegional').withColumnRenamed('SEASON', 'SEASON2').withColumnRenamed('WEEK', 'WEEK2')
CWide = traintest.where(traintest.PrevWeek=="Widespread").groupBy("SEASON","WEEK").count().withColumnRenamed('count', 'NWidespread').withColumnRenamed('SEASON', 'SEASON2').withColumnRenamed('WEEK', 'WEEK2')

traintest = traintest.join(CSpo,(traintest.SEASON == CSpo.SEASON2) &  (traintest.WEEK == CSpo.WEEK2),how = 'left').drop(CSpo.SEASON2).drop(CSpo.WEEK2)
traintest = traintest.join(CLoc,(traintest.SEASON == CLoc.SEASON2) &  (traintest.WEEK == CLoc.WEEK2),how = 'left').drop(CLoc.SEASON2).drop(CLoc.WEEK2)
traintest = traintest.join(CNoAct,(traintest.SEASON == CNoAct.SEASON2) &  (traintest.WEEK == CNoAct.WEEK2),how = 'left').drop(CNoAct.SEASON2).drop(CNoAct.WEEK2)
traintest = traintest.join(CReg,(traintest.SEASON == CReg.SEASON2) &  (traintest.WEEK == CReg.WEEK2),how = 'left').drop(CReg.SEASON2).drop(CReg.WEEK2)
traintest = traintest.join(CWide,(traintest.SEASON == CWide.SEASON2) &  (traintest.WEEK == CWide.WEEK2),how = 'left').drop(CWide.SEASON2).drop(CWide.WEEK2)

traintest.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- STATENAME: string (nullable = true)
 |-- ACTIVITYESTIMATE: string (nullable = true)
 |-- WEEK: integer (nullable = true)
 |-- SEASON: string (nullable = true)
 |-- POP: integer (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- HealthCare_All: double (nullable = true)
 |-- HealthCare_Under65: double (nullable = true)
 |-- HealthCare_Under18: double (nullable = true)
 |-- HealthCare_Over65: double (nullable = true)
 |-- INCOME: integer (nullable = true)
 |-- TypeA_subH3: integer (nullable = true)
 |-- TypeA_subH1N1: integer (nullable = true)
 |-- TypeA: integer (nullable = true)
 |-- TypeA_subH3N2: integer (nullable = true)
 |-- TypeA_subU: integer (nullable = true)
 |-- TypeA_subH1: integer (nullable = true)
 |-- TypeB: integer (nullable = true)
 |-- PrevWeek: string (nullable = true)
 |-- NSporadic: long (nullable = true)
 |-- NLocalAct: long (nullable = true)
 |-- NNoAct: long (nullable = true)
 |-- NRegional: long (nullable = tr

In [6]:
traintest.count()

16615

In [6]:
##### during how many week the flu activity haven't change for each state
# SinceAct 
print('this step take it time')
def since(table):
    l = table.collect()
    SinceAct = [0]
    for k in range(1,table.count()):
        c = 0
        i = k-1
        while (l[i]['ACTIVITYESTIMATE'] == l[k]['ACTIVITYESTIMATE']):
            c += 1
            i -= 1
            if (i < 0): 
                break
        SinceAct.append(c)
    return(SinceAct)

#traintest = traintest.withColumn('SinceAct',spark.createDataFrame((since(traintest),),['SinceAct'])['SinceAct'])
l = since(traintest)
print(len(l))
import numpy as np
np.savetxt("SinceAct.csv", l, delimiter=",")

#df = spark.createDataFrame((since(traintest),),['SinceAct']) 
#df.show(5)
#traintest.withColumn('SinceAct',since(traintest)) # Rebuild data frame
#traintest.withColumn('SincseAct',since(traintest.collect()['ACTIVITYESTIMATE']))
#traintest.show(2)

this step take it time
16615
+------------------------+
|0.000000000000000000e+00|
+------------------------+
|                     0.0|
|                     1.0|
|                     2.0|
|                     3.0|
|                     4.0|
+------------------------+
only showing top 5 rows



In [11]:
SA = spark.read.csv("SinceAct.csv", header=False, inferSchema=True)
SA = SA.withColumnRenamed('_c0','SinceAct')
SA.show(5)


+--------+
|SinceAct|
+--------+
|     0.0|
|     0.0|
|     1.0|
|     2.0|
|     3.0|
+--------+
only showing top 5 rows



In [14]:
traintest.withColumn('SinceAct',SA['SinceAct']).show(50)

AnalysisException: 'resolved attribute(s) SinceAct#1105 missing from HealthCare_Over65#75,NNoAct#262L,AREA#71,SEASON#69,HealthCare_Under65#73,POP#70,NLocalAct#219L,TypeA#79,STATENAME#66,INCOME#76,ACTIVITYESTIMATE#67,HealthCare_All#72,NWidespread#348L,WEEK#68,TypeB#83,_c0#65,TypeA_subH3#77,NRegional#305L,TypeA_subH1N1#78,TypeA_subH3N2#80,TypeA_subU#81,TypeA_subH1#82,PrevWeek#104,NSporadic#176L,HealthCare_Under18#74 in operator !Project [_c0#65, STATENAME#66, ACTIVITYESTIMATE#67, WEEK#68, SEASON#69, POP#70, AREA#71, HealthCare_All#72, HealthCare_Under65#73, HealthCare_Under18#74, HealthCare_Over65#75, INCOME#76, TypeA_subH3#77, TypeA_subH1N1#78, TypeA#79, TypeA_subH3N2#80, TypeA_subU#81, TypeA_subH1#82, TypeB#83, PrevWeek#104, NSporadic#176L, NLocalAct#219L, NNoAct#262L, NRegional#305L, ... 2 more fields];;\n!Project [_c0#65, STATENAME#66, ACTIVITYESTIMATE#67, WEEK#68, SEASON#69, POP#70, AREA#71, HealthCare_All#72, HealthCare_Under65#73, HealthCare_Under18#74, HealthCare_Over65#75, INCOM

1 1
2 2
3 3
4 3
5 3
6 3


# Tree classification

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier, DecisionTreeClassifier)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#### Formating the data

In [6]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)
from pyspark.ml.linalg import Vectors

In [7]:
State_indexer = StringIndexer(inputCol='STATENAME',outputCol='StateIndex')
State_encoder = OneHotEncoder(inputCol='StateIndex',outputCol='StateVec')

Activity_indexer = StringIndexer(inputCol='ACTIVITYESTIMATE',outputCol='ActivityIndex')
Activity_encoder = OneHotEncoder(inputCol='ActivityIndex',outputCol='ActivityVec')

Season_indexer = StringIndexer(inputCol='SEASON',outputCol='SeasonIndex')
Season_encoder = OneHotEncoder(inputCol='SeasonIndex',outputCol='SeasonVec')

In [8]:
assembler = VectorAssembler(
  inputCols=['StateVec',
             'WEEK',
             'SeasonVec',
             'POP',
             'AREA',
             'HealthCare_All',
             'HealthCare_Under65',
             'HealthCare_Under18',
             'HealthCare_Over65',
             'INCOME',
             'TypeA_subH3',
             'TypeA_subH1N1',
             'TypeA',
             'TypeA_subH3N2',
             'TypeA_subU',
             'TypeA_subH1',
             'TypeB'],
    outputCol="features")

#,
#             'TypeA_subH3',
#             'TypeA_subH1N1',
#             'TypeA',
#             'TypeA_subH3N2',
#             'TypeA_subU',
#             'TypeA_subH1',
#             'TypeB'

In [15]:
rf = RandomForestClassifier(labelCol="ActivityIndex", featuresCol="features", numTrees=20)

In [10]:
pipeline = Pipeline(stages=[State_indexer,Activity_indexer,Season_indexer,
                           State_encoder,Season_encoder,
                           assembler,rf])

In [11]:
(train, test) = traintest.randomSplit([0.9, 0.1])

In [16]:
fit_model = pipeline.fit(train)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1035, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o196.fit

In [ ]:
prediction_rf = fit_model.transform(test)


In [ ]:
prediction_rf.show(5)

In [14]:
evaluator = MulticlassClassificationEvaluator(labelCol="ActivityIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction_rf)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.45109
